In [ ]:
from header import *
from scipy import optimize
import matplotlib.ticker as mtick
plt.rcParams.update({'font.size': 16})
plots_dir = Path('imgs/')

In [ ]:
def plot_scaling(df, x, y, filename, trend_line='', xlog=False, ylog=False, title=None, split='alg', fit_min=None, fit_max=None, cone_exp=1, cone=None, cone_x=10**4, labels=True, show_fit=True, ls='-'):
    #fig, ax = plt.subplots()
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(6, 4, forward=True)
    
    if not isinstance(split, list):
        split = [split]

    def group_label(algo):
        if len(split) == 1: return ''
        if split[1] in ['r']:
            if np.isnan(algo[1]):
                return ''
            if int(algo[1])==1:
                return f' (exact)'
            else:
                return f' (inexact)'
            #return f' (r={int(algo[1])})'
 
    # Pivot table
    d = df.pivot(index=x, columns=split, values=[y, 'r'])

    # PLOT DATA
    for algo in d[y].columns:
        key = algo[0] if isinstance(algo, tuple) else algo
        marker = r2marker(key, d['r'][algo][d[y][algo].index[0]])
        d[y][algo].plot(ax=ax, alpha=1, zorder=3, rot=0, color=algo2color(key), marker=marker, ls=ls, legend=False)

    # DRAW CONE
    # Fills the region between x**cone and x**(cone+1)
    def draw_cone(x_origin, x_max=d.index.max()):
        # draw cone
        x_max *= 3
        y_min = d.index.min()
        data = d[(y, cone)]
        if len(split) > 1 and split[1] == 'r':
            data = data[1]
        y_origin = data[x_origin]
        coef = (data.iloc[-1] - y_origin) / (data.index[-1] - x_origin)  # tan
        x_cone      = (x_origin, x_max)
        y_cone_lin = (y_origin, y_origin * (x_max / x_origin)**(cone_exp))
        y_cone_quad = (y_origin, y_origin * (x_max / x_origin)**(cone_exp+1))
        ax.fill_between(x_cone, y_cone_lin, y_cone_quad, color='grey', alpha=0.15)
    
    if cone is not None:
        draw_cone(x_origin=cone_x)
    
    # FIT y = x^C
    if trend_line == "poly":
        z = {}
        for algo in d[y].columns:
            s = d[y][algo].dropna()
            if fit_min:
                s = s[s.index >= fit_min(algo)]
            if fit_max:
                s = s[s.index <= fit_max(algo)]
            #s = s[s>0]
            if len(s) > 1:
                z[algo] = np.polyfit(np.log(s.index), np.log(s), 1)
        xs = list(d.index)
        
        # Best fit lines
        exps = {}
        for algo in z:
            key = algo[0] if isinstance(algo, tuple) else algo
            regression_line = []
            a, b = z[algo]
            plot_xs = []
            for i in xs:
                # fit only on points >= fit_min(algo)
                if fit_min and i < fit_min(algo):
                    continue
                if fit_max and i > fit_max(algo):
                    continue
                if i > d[y][algo].dropna().index.max():
                    continue
                plot_xs.append(i)
                regression_line.append((i**a) * np.exp(b))
            
            label = ''
            if len(d[y][algo].dropna()) > 1:
                if show_fit:
                    ax.plot(plot_xs, regression_line, linestyle='-', color=algo2color(key), alpha=0.9)
                label = '$\sim n^{{{:0.2f}}}$'.format(a)  ## np.exp(b)*x^a
                exps[algo] = f'{a:.2f}'
            ax.text(plot_xs[-1], regression_line[-1], algo2beautiful(key) + group_label(algo) + label,
                    color=algo2color(key), ha='right', va='bottom', size=15, alpha=1)
        print(exps)
    elif trend_line:
        print(trend_line)
        assert(False)
    else:
        if labels:
            label_x = d[y].index[-1]
            for algo in d[y].columns:
                key = algo[0] if isinstance(algo, tuple) else algo
                ax.text(label_x, d[y][algo][label_x], algo2beautiful(key) + group_label(algo),
                        color=algo2color(key), ha='right', va='bottom', size=15, alpha=1)
    
    
    # ENABLE LOG SCALE
    if ylog:
        ax.set_yscale('log')
    else:
        ax.set_ylim(0)
    
    if xlog:
        ax.set_xscale('log')
        
    # SET LIMITS FOR LOG AXES
    if xlog:
        ax.set_xlim(1/1.5*d.index.min(), 1.5*d.index.max())
    if ylog:
        ax.set_ylim(1/3*d.min().min(), 3*d.max().max())
        
    # Background
    ax.set_facecolor('#F8F8F8')
    
    # No border
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if x not in ['e', 'e_pct']:
        ax.spines['left'].set_visible(False)
        
    # GRID: major y-axis
    ax.grid(False, axis='x', which='major')
    ax.grid(False, axis='x', which='minor')
    ax.grid(True, axis='y', which='major', color='w')
    ax.grid(False, axis='y', which='minor')
    
    # Ticks, no minor ticks
    ax.tick_params(
        axis='both',          # changes apply to the x-axis
        which='minor',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        left=False,right=False,
        labelbottom=False # labels along the bottom edge are off
    )
    if x == 'n':
        ax.set_xticks(list(filter(lambda x: d.index.min() <= x and x <= d.index.max(), [10**e for e in range(10)])))
    if x == 'e_pct':
        ax.xaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
        ax.margins(x=0)
        ax.set_xlim(left=0)
    #if y == 's_per_pair':
    #    ax.set_yticks(list(filter(lambda x: d.min().min() <= x and x <= d.max().max(), [10**e for e in range(-10, 10, 2)])))
    #if y == 's_per_bp':
    #    ax.set_yticks(list(filter(lambda x: d.min().min() <= x and x <= d.max().max(), [10**e for e in range(-10, 10, 1)])))
    
    # axis labels
    ax.set_xlabel(col2name(x), size=18)  # weight='bold',
    ax.set_ylabel(col2name(y), rotation=0, ha='left', size=18)
    ax.yaxis.set_label_coords(-0.10,1.00)
    
    if title:
        ax.set_title(title)
    
    plt.savefig(plots_dir/(filename+'.pdf'), bbox_inches='tight')

In [ ]:
# SPEEDUP [Tools, scaling] -- time/alignment
df = read_benchmarks('table/tools_N1e7.tsv')
df = df[df.exit_status == "ok"]
df = df[df.n >= 3*10**3]
for e in pd.unique(df.e):
    df_n = df[df.e == e]
    plot_scaling(df_n, y='s_per_pair', x='n', filename=f'tools_e{e}', xlog=True, ylog=True, trend_line='poly', show_fit=True, title=None, cone='csh', cone_x=3*10**4, ls='')

In [ ]:
# SCALING WITH N
 
df = read_benchmarks('table/scaling_n_N1e7.tsv')
plot_scaling(df, y='s_per_pair', x='n', split=['alg', 'r'], filename='scaling_n', ylog=True, xlog=True, cone='cp-csh', cone_x=100, trend_line = 'poly', ls='')

In [ ]:
# SCALING WITH E
df = read_benchmarks('table/scaling_e_N1e6.tsv')
plot_scaling(df, y='s_per_pair', x='e_pct', split=['alg', 'r'], filename=f'scaling_e', ylog=False)

In [ ]:
# TABLE AT 5%, 10^7
df = read_benchmarks('table/tools_N1e7.tsv')
df['alg_order'] = pd.Categorical(
    df['alg'],
    categories=['edlib','biwfa','sh','csh'],
    ordered=True
)
df = df[(df.e <= 0.05) & (df.n == 10**7)]
pt = df.pivot_table(['s_per_pair','max_uss'],['alg_order'], ['e'])
pt = pt[['s_per_pair', 'max_uss']]
display(pt)

# Speedup
times = pt['s_per_pair'][0.05]
our_best = min(times['sh'], times['csh'])
their_best = min(times['edlib'], times['biwfa'])
print(f"MAX SPEEDUP {their_best:.4}/{our_best:.4}={their_best/our_best:.4}")

In [ ]:
# MAX MEMORY
df = read_benchmarks('table/tools_N1e7.tsv')
df = df[df.alg.isin(['sh', 'csh'])]
df = df.groupby(['e', 'alg'])['max_uss'].agg('max')
display(df)

In [ ]:
# SCALING WITH K

df = read_benchmarks('table/scaling_k_N1e7.tsv')
algs=['cp-sh','cp-csh']
ns = [10**3, 10**4, 10**5, 10**6, 10**7]
df = df[df.alg.isin(algs)]
df = df[df.n.isin(ns)]

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
d = df.pivot(index='k', columns=['alg', 'm', 'n'], values='s_per_pair')
for alg in algs:
    d[alg].plot(ax=ax, alpha=0.6, color=algo2color(alg), marker=algo2marker(alg), ls='-', legend=False)
ax.set_xticks(pd.unique(df.k))
ax.set_yscale('log')
plt.savefig(plots_dir/('scaling_k.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
# SCALING WITH K (over e)

df = read_benchmarks('table/scaling_k_n_N1e7.tsv')
algs=['cp-sh','cp-csh']
df = df[df.alg.isin(algs)]
es = [0.02, 0.06, 0.10, 0.14]
df = df[df.e.isin(es)]

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
d = df.pivot(index='k', columns=['alg', 'm', 'e'], values='s_per_pair')
for alg in algs:
    d[alg].plot(ax=ax, alpha=0.6, color=algo2color(alg), marker=algo2marker(alg), ls='-', legend=False)
ax.set_xticks(pd.unique(df.k))
plt.savefig(plots_dir/('scaling_k_e_lin.pdf'), bbox_inches='tight')
plt.show()


fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
d = df.pivot(index='k', columns=['alg', 'm', 'e'], values='s_per_pair')
for alg in algs:
    d[alg].plot(ax=ax, alpha=0.6, color=algo2color(alg), marker=algo2marker(alg), ls='-', legend=False)
ax.set_xticks(pd.unique(df.k))
ax.set_yscale('log')
plt.savefig(plots_dir/('scaling_k_e_log.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
# MEMORY
name = "memory"
df = tools
# Only print things that didn't time out.
#df = df[df.exit_status == "ok"]
df = df[df.alg != 'cp-csh+gap']
df = df[df.alg != 'csh+gap']
# The constant memory of 30MB (snakemake error?) is too large in small tests
df = df[df.n >= 10**4]
def fit_from(algo):
    if algo in ['dijkstra', 'dijkstra_nogreedy', 'pa_noprune', 'pa_noprune_nogreedy']: return 10**4
    return 10**5

for e in pd.unique(df.e):
    df_n = df[df.e == e]
    df_n = df_n[df_n.max_uss != '-']
    df_n = df_n[df_n.max_uss > 0.025]
    if df_n.empty: continue
    plot_scaling(df_n, y='max_uss', x='n', filename=f'{name}_e{e}', xlog=True, ylog=True, trend_line='poly', title=f'Error rate {int(100*e)}%'.format(e), fit_min=fit_from)

In [ ]:
# Expanded states
name = "expanded"
df = tools
df = df[df.alg != 'cp-csh+gap']
df = df[df.alg != 'csh+gap']

# Only print things that didn't time out.
df = df[df.exit_status == "ok"]
def fit_from(algo):
    if algo in ['dijkstra', 'pa_noprune_nogreedy']:
        return 10000
    return 10000
import math

# Table of band
b = df[['alg', 'n', 'e', 'band']].dropna()
alg_order = ['sh', 'csh', 'csh-noprune',  'dijkstra']
b['alg_idx'] = b['alg'].map(lambda a: alg_order.index(a))
b = b.sort_values(by=['e', 'n', 'alg_idx'])
display(pd.pivot_table(b, values='band', columns=['n'], index=['e', 'alg'], sort=False))

# Plots of expanded states
for e in pd.unique(df.e):
    df_n = df[df.e == e]
    df_n = df_n.dropna(subset=['expanded'])
    plot_scaling(df_n, y='expanded', x='n', filename=f'{name}_e{e}', xlog=True, ylog=True, trend_line='poly', title=f'Error rate {int(100*e)}%', fit_min=fit_from, cone='csh')

In [ ]:
# MAX MEMORY USAGE

df = tools
df = df[df.n >= 1000000]
df = df[df.e >= 0.1]
df = df.sort_values(by=['alg'])
display(df[['alg', 'n', 'e', 's', 'max_uss']])

In [ ]:
# BEST PARAMS
p = params
p = p[p.e == 0.1]
p = p[p.alg == 'cp-sh']
p = p[p.n == 1000000]
display(p) # all
